## Problem Case 1: Cut Rod probelm (Review)
1. Give functions which can give the value generated by the highest value cut method and the cut solution.
2. Give optimized method

In [63]:
# 1m=1RMB, 2m=5rmb,3m=8rmb ...
original_price = [1, 5, 8, 9, 10, 17, 17, 20, 24, 30, 35]
from collections import defaultdict
price = defaultdict(int)
for i,p in enumerate(original_price):
    price[i+1] = p

## Get the max splitting by enumerate

In [100]:
def r(n):#Recursive
    
    return max(
        [price[n]] + [r(i) + r(n-i) for i in range(1,n)]
    )
#10m
#max(price[10],r[1]+r[9],r[2]+r[8],...,r[9]+r[1])

#[1]+[2]=[1,2]

In [70]:
r(15)

45

In [71]:
import time

In [72]:
#@get_time
def fibonacci(n):
    if n  <= 2:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)

In [73]:
start = time.time()
print(fibonacci(34))
end = time.time()
print(end-start)

5702887
2.091911792755127


In [74]:
mem = defaultdict()
def fibonacci_op(n):
    if n in mem:#check whether number is in the memory
        return mem[n]
    else: 
        if n <= 2:
            mem[n] = 1
            return n
        else:
            result = fibonacci_op(n-1) + fibonacci_op(n-2)
            mem[n] = result
            return result

In [75]:
start = time.time()
print(fibonacci_op(64))
end = time.time()
print(end-start)

14662949395604
0.0010039806365966797


# Analysis: How to optimize

## A Simpler Problem

### Decorator

In [76]:
def get_time(func):
    def wrapper(*args):
        start = time.time()
        func(*args)
        end = time.time()
        print('used time : {}'.format(end-start))
    return wrapper

In [77]:
def f1(func):
    def wrapper(*args,**kwargs):
        print('Started')
        func(*args,**kwargs)
        print('Ended')
    return wrapper

In [78]:
def f():
    print('HELLO')

In [79]:
f = f1(f)

In [80]:
print(f.__name__)

wrapper


In [81]:
@f1 #装饰器
def g(a):
    print(a)

In [82]:
g('hello')

Started
hello
Ended


In [83]:
def k(*arg,**kwargs):
    print(kwargs)

In [84]:
k(6,b=5)

{'b': 5}


In [101]:
from functools import wraps

In [102]:
def memo(f):
    memo.already_computed = {}
    @wraps(f)
    def _wrap(arg):
        if arg in memo.already_computed:
            result = memo.already_computed[arg]
        else:
            result = f(arg)
            memo.already_computed[arg] = result
        return result
    return _wrap
#加了装饰器后，能减少重复计算

## We use this method to solve Cut Rod probelm

In [103]:
solution = {}

In [104]:
#利用装饰器@memo记录缓存
@memo
def r_op(n):
    """
    Args: n is the iron length
    Return: the max revenue 
    """
    max_price, max_split = max(
        [(price[n], 0)] + [(r_op(i) + r_op(n-i), i) for i in range(1, n)], key=lambda x: x[0]
    )

    solution[n] = (n - max_split, max_split)
    
    return max_price

In [105]:
r_op(20)

60

In [106]:
price

defaultdict(int,
            {1: 1,
             2: 5,
             3: 8,
             4: 9,
             5: 10,
             6: 17,
             7: 17,
             8: 20,
             9: 24,
             10: 30,
             11: 35,
             15: 0,
             14: 0,
             13: 0,
             12: 0,
             20: 0,
             19: 0,
             18: 0,
             17: 0,
             16: 0})

In [107]:
solution

{1: (1, 0),
 2: (2, 0),
 3: (3, 0),
 4: (2, 2),
 5: (3, 2),
 6: (6, 0),
 7: (6, 1),
 8: (6, 2),
 9: (6, 3),
 10: (10, 0),
 11: (11, 0),
 12: (11, 1),
 13: (11, 2),
 14: (11, 3),
 15: (13, 2),
 16: (14, 2),
 17: (11, 6),
 18: (17, 1),
 19: (17, 2),
 20: (17, 3)}

## How do we parse solution?

In [108]:
def parse_solution(n):
    left_split, right_split = solution[n]
    
    if right_split == 0: return [left_split]
    
    return parse_solution(left_split) + parse_solution(right_split)

In [109]:
r_op(24)

75

In [110]:
parse_solution(20)

[11, 6, 3]

## Problem Case 2: Edit Distance
If there are two strings with edit distance, give the parse solution for string 1 to string 2 

In [135]:
solution = {}

In [136]:
from functools import lru_cache

In [164]:
@lru_cache(maxsize=2**10)
def edit_distance(string1, string2):
    
    if len(string1) == 0: return len(string2)
    if len(string2) == 0: return len(string1)
    
    tail_s1 = string1[-1]
    tail_s2 = string2[-1]
    
    candidates = [
        (edit_distance(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)),  
        # string 1 delete tail
        (edit_distance(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2)),  
        # string 1 add tail of string2
    ]
    
    if tail_s1 == tail_s2:
        min_distance, operation = min((edit_distance(string1[:-1], string2[:-1]) + 0, '-'),
                                      (edit_distance(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)),  
        # string 1 delete tail
                                      (edit_distance(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2)),  
        # string 1 add tail of string2
                                      key=lambda x: x[0])
        solution[(string1, string2)] = operation 
        return min_distance
        
    else:
        min_distance, operation = min((edit_distance(string1[:-1], string2[:-1]) + 1, 'SUB {} => {}'.format(tail_s1, tail_s2)),
                                      (edit_distance(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)),  
        # string 1 delete tail
                                      (edit_distance(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2)),  
        # string 1 add tail of string2
                                      key=lambda x: x[0])
        solution[(string1, string2)] = operation 
        return min_distance

def parse_operations(string1, string2):
    if len(string1) == 0 and len(string2) == 0:
        return
    elif len(string1) == 0:
        print('{}->{}: ADD {}'.format(s1, s2, s2))
        return
    elif len(string2) == 0:
        print('{}->{}: DEL {}'.format(s1, s2, s1))
        return
        
    operation = solution[(string1, string2)] 
    print('{}->{}: {}'.format(string1, string2, operation))
    if '-' in operation:
        return parse_operations(string1[:-1], string2[:-1])
    elif 'SUB' in operation:
        return parse_operations(string1[:-1], string2[:-1])
    elif 'ADD' in operation:
        return parse_operations(string1, string2[:-1])
    elif 'DEL' in operation:
        return parse_operations(string1[:-1], string2)

In [167]:
string1 = 'ABCDECG'
string2 = 'ABCCEF'
print('Edit distance: ',edit_distance(string1, string2))
print('Solution: ')
parse_operations(string1, string2)

Edit distance:  3
Solution: 
ABCDECG->ABCCEF: SUB G => F
ABCDEC->ABCCE: DEL C
ABCDE->ABCCE: -
ABCD->ABCC: SUB D => C
ABC->ABC: -
AB->AB: -
A->A: -


# Problem Case 3: Pinyin Auto Correction Problem

In [1]:
chinese_dataset = 'article9k.txt'

In [2]:
CHINESE_CHARATERS = open(chinese_dataset,encoding='utf-8').read()

In [3]:
CHINESE_CHARATERS[:30]

'此外自本周6月12日起除小米手机6等15款机型外其余机型已暂'

In [4]:
import pinyin

In [5]:
pinyin.get('你好，中国',format='strip',delimiter=' ')

'ni hao ， zhong guo'

In [6]:
def chinese_to_pinyin(character):
    return pinyin.get(character, format='strip', delimiter=' ')

In [7]:
CHINESE_CHARATERS_COPYS = chinese_to_pinyin(CHINESE_CHARATERS)

In [8]:
len(CHINESE_CHARATERS_COPYS)

27210499

In [9]:
import re

In [10]:
def tokens(text):
    "List all the pinyin characters"
    return re.findall('[a-z]+',text.lower())

In [11]:
CHINESE_CHARATERS_COPYS[:100]

'ci wai zi ben zhou 6 yue 1 2 ri qi chu xiao mi shou ji 6 deng 1 5 kuan ji xing wai qi yu ji xing yi '

In [12]:
tokens(CHINESE_CHARATERS_COPYS[:100])

['ci',
 'wai',
 'zi',
 'ben',
 'zhou',
 'yue',
 'ri',
 'qi',
 'chu',
 'xiao',
 'mi',
 'shou',
 'ji',
 'deng',
 'kuan',
 'ji',
 'xing',
 'wai',
 'qi',
 'yu',
 'ji',
 'xing',
 'yi']

In [13]:
from collections import Counter, defaultdict

In [14]:
import platform
platform.architecture()

('32bit', 'WindowsPE')

In [15]:
PINYIN_COUNT = Counter(tokens(CHINESE_CHARATERS_COPYS))

In [16]:
PINYIN_COUNT.most_common(20)

[('shi', 216323),
 ('de', 198649),
 ('yi', 154920),
 ('ji', 129319),
 ('zhi', 98571),
 ('zhong', 80052),
 ('wei', 79894),
 ('guo', 75090),
 ('li', 73068),
 ('you', 72500),
 ('bu', 70755),
 ('qi', 65829),
 ('zai', 64103),
 ('jin', 63266),
 ('yu', 61941),
 ('ren', 60081),
 ('he', 59592),
 ('xing', 59328),
 ('gong', 58619),
 ('ye', 57979)]

In [17]:
len(PINYIN_COUNT)

419

In [18]:
def correct(word):
    'Find the most possible pinyin based on edit distance'
    # Prefer edit distance 0, then 1, then 2; otherwist default to word itself
    candidates = (known(edits0(word)) or
                  known(edits1(word)) or
                  known(edits2(word)) or
                  [word])
    return max(candidates,key=PINYIN_COUNT.get)

In [19]:
def known(words):
    'Return the pinyin in our data'
    return {w for w in words if w in PINYIN_COUNT}

def edits0(word):
    'Return all strings that are zero edits away from word (i.e., just word itself).'
    return {word}

def edits2(word):
    'Return all strings that are two edits away from this pinyin.'
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)}

In [20]:
alphabet = 'abcdefghijklmnopqrstuvwxyz'

def splits(word):
    'Return a list of all possible (first, rest) pairs that comprise pinyin.'
    return [(word[:i], word[i:])
           for i in range(len(word)+1)]

def edits1(word):
    'Return all strings that are one edit away from this pinyin.'
    pairs = splits(word)
    deletes = [a+b[1:] for (a,b) in pairs if b]
    transposes = [a+b[1]+b[0]+b[2:] for (a,b) in pairs if len(b) > 1]
    replaces = [a+c+b[1:] for (a,b) in pairs for c in alphabet if b]
    inserts = [a+c+b for (a,b) in pairs for c in alphabet]
    return set(deletes + transposes + replaces + inserts)

In [21]:
splits('pinyin')

[('', 'pinyin'),
 ('p', 'inyin'),
 ('pi', 'nyin'),
 ('pin', 'yin'),
 ('piny', 'in'),
 ('pinyi', 'n'),
 ('pinyin', '')]

In [22]:
print(edits0('pinyin'))

{'pinyin'}


In [23]:
print(edits1('pinyin'))

{'xpinyin', 'pynyin', 'pinqyin', 'pinyinh', 'pinyn', 'lpinyin', 'pinyiin', 'pinxin', 'pinyiw', 'pinyiy', 'pinyign', 'pinfyin', 'pilnyin', 'pinypin', 'pfnyin', 'pinyein', 'pianyin', 'pinqin', 'rinyin', 'hinyin', 'pinynin', 'pinykn', 'pinycin', 'pinyzn', 'kpinyin', 'zpinyin', 'pinypn', 'pnnyin', 'pinyii', 'pinydin', 'pinycn', 'kinyin', 'pcnyin', 'ninyin', 'ainyin', 'pinyian', 'ipinyin', 'npinyin', 'plnyin', 'pioyin', 'pinyidn', 'punyin', 'psnyin', 'pinybn', 'ptnyin', 'pwinyin', 'pizyin', 'pinyien', 'pqnyin', 'pinkyin', 'pinyi', 'picyin', 'yinyin', 'pjinyin', 'pinyinv', 'pyinyin', 'piwnyin', 'pinyinl', 'inyin', 'picnyin', 'piznyin', 'panyin', 'pinykin', 'pinuin', 'pixyin', 'pinsyin', 'pinyic', 'pxinyin', 'dinyin', 'pinyini', 'qpinyin', 'pinyine', 'pmnyin', 'opinyin', 'pxnyin', 'pwnyin', 'pingin', 'pinyiun', 'pznyin', 'jinyin', 'pinrin', 'epinyin', 'pinhyin', 'pinyinb', 'pnyin', 'pinyfin', 'winyin', 'gpinyin', 'pinybin', 'pinbin', 'pinytn', 'pinyni', 'pinkin', 'pinyln', 'pijyin', 'binyin',

## Test

In [24]:
correct('yin')

'yin'

In [25]:
correct('yign')

'ying'

In [26]:
correct('yign')

'ying'

In [27]:
def correct_sequence_pinyin(text_pinyin):
    return ' '.join(map(correct, text_pinyin.split()))

In [28]:
correct_sequence_pinyin('zhe sih yi ge pign gue')

'zhe shi yi ge pin guo'

In [29]:
correct_sequence_pinyin('wo aio yu yan cho li')

'wo ai yu yan chu li'

# Homework？    
#### How to complete automatic trimming without spaces? -> How to complete the automatic segmentation of Pinyin?   
###### Tip：The language model mentioned in the first lesson

woyaoshangqinghua
w yaoshangqinghua
wo yaoshangqinghua
woyao shangqinghua

-> DP
先可以分成多种情况，如上，成为dp问题；然后用ngram看哪个概率最大；最后用correct——squence—— pinyin

In [1]:
import re
import pinyin
from collections import Counter
from functools import lru_cache

file_path = 'article9k.txt'

In [2]:
class PinyinCouter:
    """Provide the operations to counter in Chinese-Pinyin."""
    def __init__(self, dataset, counters={}):
        """Create the obj with two attributes.
        :param dataset: The file path
        :param counter: Defaul couter by a blank dict
        """
        self._dataset = dataset
        self._counter = counters

    @property
    def get_counter(self):
        return self._counter

    def read_file(self):
        with open(self._dataset, encoding='utf-8-sig') as f:
            CHINESE_CHARATERS = f.read()
        filter_list = re.findall('[^[a-zA-Z0-9]+', CHINESE_CHARATERS.replace('\n', ""))
        filter_text = "".join(filter_list)
        return filter_text  # you can try filter_text[:100000] to increase the spped, but sacrify a bit accuracy

    @property
    def pinyin_pair(self):
        return pinyin.get(self.read_file(), format="strip", delimiter=" ")

    def pinyin_counter(self):
        pinyin_list = self.pinyin_pair.split(" ")
        self._counter = Counter(pinyin_list)

    def expand_counter(self, n):
        """Expand the counter to 2-gram, 3-gram, 4-gram... till n-gram.
        :param n: the ngram model number
        :return: a final big counter
        """
        final_counter = Counter()
        for i in range(2, n+1):
            expand_list = generate_ngrams(self.pinyin_pair, i)
            final_counter += Counter(expand_list)
        return self.get_counter + final_counter

In [3]:
class EditDistance:
    """Provide the operation to calculate the edit distance.
    :Attribute alphabet: the Letter string between a-z.
    """
    alphabet = 'abcdefghijklmnopqrstuvwxyz'
    def __init__(self):
        pass

    def edits0(self, word):
        """Return all strings that are zero edits away from word (i.e., just word itself)."""
        return {word}

    def edits2(self, word):
        """Return all strings that are two edits away from this pinyin."""
        return {e2 for e1 in self.edits1(word) for e2 in self.edits1(e1)}

    def edits3(self, word):
        """Return all strings that are three edits away from this pinyin."""
        return {e3 for e2 in self.edits2(word) for e3 in self.edits1(e2)}

    def edits1(self, word):
        """Return all strings that are one edit away from this pintin"""
        pairs = self.splits(word)
        deletes = EditDistance.get_deletes(pairs)
        transposes = EditDistance.get_transposes(pairs)
        replaces = EditDistance.get_replaces(pairs)
        inserts = EditDistance.get_inserts(pairs)
        return set(deletes + transposes + replaces + inserts)

    @staticmethod
    def get_deletes(pairs):
        return [a + b[1:] for (a, b) in pairs if b]

    @staticmethod
    def get_transposes(pairs):
        return [a + b[1] + b[0] + b[2:] for (a, b) in pairs if len(b) > 1]

    @staticmethod
    def get_replaces(pairs):
        return [a + c + b[1:] for (a, b) in pairs for c in EditDistance.alphabet if b]

    @staticmethod
    def get_inserts(pairs):
        return [a + c + b for (a, b) in pairs for c in EditDistance.alphabet]

    def splits(self, word):
        """Return a list of all possible (first, rest) pairs that comprise pinyin."""
        return [(word[:i], word[i:]) for i in range(len(word) + 1)]

In [4]:
def generate_ngrams(sentence, n):
    """Build the ngram model by two different method.
       Fun1: Use the zip function to help us generate n-grams (faster)
       Fun2: Use List generation (slower)
    :param sentence: the initial string
    :param n: gram model number
    :return:
    """
    # length = len(sentence.split(" "))
    tokens = [token for token in sentence.split(" ") if token != ""]
    ngrams = zip(*[tokens[i:] for i in range(n)])
    # ngrams = [tokens[i: n+i] for i in range(length) if n+i <= length]
    return ["".join(ngram) for ngram in ngrams]

In [5]:
def known(words, counter):
    """Find the words exit in counter.
    :param words: should be a list
    :param counter: background counter
    :Return: the pinyin words we have noticed.
    """
    return {w for w in words if w in counter}

In [6]:
def correct(word, counter, ed):
    """Find the most possible pinyin based on edit distance.
    :param word: The incorrect sentence or word
    :param counter: The base counter data to correct the word
    :param ed: The EditDistance obj
    :return: A most likely word after correction
    """
    # Prefer edit distance 0, then 1, then 2; otherwise default to word itself.
    candidates = (known(ed.edits0(word), counter) or
                  known(ed.edits1(word), counter) or
                  known(ed.edits2(word), counter) or
                  [word]
                  )
    return max(candidates, key=counter.get)  # return the most possible

In [7]:
# To split the string after correction
def P(string_list, counter, default=0):
    """Get the probability of a list of strings depend on Thesaurus.
    :param string_list: The origin semantics list
    :param default: The origin probability
    :param count: The Couter for search
    :return: A chance number less than 1
    """
    probility = 1
    for token in string_list:
        if token not in counter:
            probility *= default
        else:
            probility *= counter.get(token) / sum(counter.values())
    return probility

In [8]:
# some global variables
notes = []
split_solutions = {}

count1 = PinyinCouter(file_path)
count1.pinyin_counter()
counter = count1.get_counter


@lru_cache(maxsize=2 ** 10)
def best_split(string):
    """split a string to a few words into the dict according to the max probability.
    :param string: the phrase to be cut
    :return: the most likely percentage
    """
    # global counter
    # global notes
    # global split_solutions
    notes = [(P([string], counter), '', string)] + [(best_split(string[:i]) * best_split(string[i:]),
                                                     string[:i], string[i:]) for i in range(1, len(string))]
    prob, left, right = max(notes, key=lambda x: x[0])

    split_solutions[string] = (left, right)
    return prob


def parse_split_solution(string):
    """Operate the split function.
    :param string: the phrase without splitting
    :return: a complete phrase after split
    """
    left, right = split_solutions[string]
    if not left: return [right]
    return parse_split_solution(left) + parse_split_solution(right)


def mixin(demo, max_counter, edit_op):
    """Integrate the correction and split operation.
    :param demo: the original phrase
    :param max_counter: the background counter
    :param edit_op: param for correct function
    :return: the final right phrase
    """
    fix_text = correct(demo, max_counter, edit_op)
    prob = best_split(fix_text)
    return parse_split_solution(fix_text)

In [13]:
# testing the function
edit_op = EditDistance()
final_counter3 = count1.expand_counter(5)

In [14]:
sample3 = "xiaopegnyuo" 
correct_sample3 = mixin(sample3, final_counter3, edit_op)
print(correct_sample3)

['xiao', 'peng', 'you']


In [15]:
del correct_sample3

In [16]:
sample1 = "qignhuadaxeu"  
    
correct_sample1 = mixin(sample1, final_counter3, edit_op)
print(correct_sample1)

['qing', 'hua', 'da', 'xue']


In [17]:
del correct_sample1

In [55]:
sample2 = "tiantinaxinagshang"  
    
correct_sample2 = mixin(sample2, final_counter3, edit_op)
print(correct_sample2)

['tian', 'tian', 'xiang', 'shang']
